In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import urllib
import requests

In [5]:
searchlink = 'http://www.chemspider.com/Search.aspx?q='
def bsoupReq(drugname):
    req = urllib.request.Request(searchlink+drugname,
                                     headers={'User-Agent' : "Magic Browser"}) 
    con = urllib.request.urlopen( req )
    html = bs(con,'html.parser')
    return html

In [7]:
spage = bsoupReq('Acacetin')

In [39]:
def checkCmpdSrc(soupPage,drugname):
    try:
        lstDetailSection=[]
        for j in soupPage.findAll('h2',{'class':'section-head'}):
            lstDetailSection.append(j.find('span').text)
        return ('Compound Source:' in lstDetailSection)
    except Error as e:
        print(e)

In [24]:
def classifyDrg(drugname):
    bsoupreq = bsoupReq(drugname)
    return checkCmpdSrc(bsoupreq,drugname)

In [41]:
classifyDrg('taxol')

True

In [42]:
classifyDrg('abilify')

False

In [109]:
payload = {'CompoundName': 'taxol','search_by_property':'SEARCH'}
npass="http://bidd2.nus.edu.sg/NPASS/search.php"

In [110]:
data = urllib.parse.urlencode(payload)
data = data.encode('ascii') # data should be bytes
req = urllib.request.Request(npass, data)

In [111]:
con = urllib.request.urlopen( req )
bs(con,'html.parser')

<html>
<head>
<title>NPASS search</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="assets/css/main.css" rel="stylesheet"/>
<script src="./jquery_plugins/tabbed-content/js/jquery-1.9.1.min.js"></script>
<script src="./jquery_plugins/tabbed-content/js/modernizr.js"></script>
<script src="./jquery_plugins/tabbed-content/js/tabs.js"></script>
<script src="./jquery_plugins/JSME_2017-02-26/jsme/jsme.nocache.js"></script>
<script>
	 //this function will be called after the JavaScriptApplet code has been loaded.
	    function jsmeOnLoad() {
	        jsmeApplet = new JSApplet.JSME("jsme_container", "480px", "400px");
	   }
	</script>
<style type="text/css">
		table, th, td {
		    border: 0;
		}
		table, td {
			height: 40;
		}
		input {
			height: 30;
		}
		#targetType ul {
		  list-style: none;
		  text-align: left;
		  padding-top: 10px;
		  padding-left:0;
		  margin: 0;
		  float: left;
		}
		#targetType li {
		  font-family: 'Oswald', sans-serif;
	

In [92]:
print(t.url)

http://bidd2.nus.edu.sg/NPASS/search.php/?CompoundName=taxol&search_by_property=SEARCH


In [112]:
t = requests.post(npass, params=payload)

In [116]:
t.content

b'<html>\r\n<head>\r\n\t<title>NPASS search</title>\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1" />\r\n\t<link rel="stylesheet" href="assets/css/main.css">\r\n\t<script src="./jquery_plugins/tabbed-content/js/jquery-1.9.1.min.js"></script>\r\n\t<script src="./jquery_plugins/tabbed-content/js/modernizr.js"></script>\r\n\t<script src="./jquery_plugins/tabbed-content/js/tabs.js"></script>\r\n\r\n\r\n\t<script src="./jquery_plugins/JSME_2017-02-26/jsme/jsme.nocache.js"></script>\r\n\r\n\t<script>\r\n\t //this function will be called after the JavaScriptApplet code has been loaded.\r\n\t    function jsmeOnLoad() {\r\n\t        jsmeApplet = new JSApplet.JSME("jsme_container", "480px", "400px");\r\n\t   }\r\n\t</script>\r\n\r\n\t<style type="text/css">\r\n\t\ttable, th, td {\r\n\t\t    border: 0;\r\n\t\t}\r\n\t\ttable, td {\r\n\t\t\theight: 40;\r\n\t\t}\r\n\t\tinput {\r\n\t\t\theight: 30;\r\n\t\t}\r\n\t\t#targetType ul {\r\n\t\t  list-style: none;\r\n\t\t  text-ali

In [11]:
url = 'http://bidd2.nus.edu.sg/NPASS/search.php'
def selenNPASS(cmpdName):
    driver = webdriver.Firefox(executable_path=r'C:/Program Files/Mozilla Firefox/geckoDriver/geckodriver.exe') 
    driver.get(url)
    cmpdname = driver.find_element_by_name("CompoundName")
    cmpdname.send_keys(cmpdName)
    search_btn = driver.find_element_by_name('search_by_property')
    search_btn.send_keys(Keys.RETURN)
    #ActionChains(driver).key_down(Keys.CONTROL).send_keys('2').key_up(Keys.CONTROL).perform()
    windows = driver.window_handles
    driver.switch_to.window(windows[1])
    try:
        element = WebDriverWait(driver, 1).until(lambda driver: matchCnt(driver)>0 )
        if element:
            return True
    except TimeoutException:
        print('{} was not found in NPASS DB'.format(cmpdName))
        return False
    finally:
        driver.close()
        driver.switch_to.window(windows[0])
    #innerHTML = driver.execute_script("return document.body.innerHTML")
    #bs(innerHTML,'html.parser')

In [3]:
def matchCnt(driver):
    matches = driver.find_element_by_tag_name("h2").text
    match_count = int(matches.split(':')[1].strip())
    print("{} matches were found".format(match_count))
    return match_count

In [12]:
selenNPASS('Adenine')

Adenine was not found in NPASS DB


False